In [15]:
import json
import time

import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import lxml
import pandas as pd
from openai import OpenAI, RateLimitError
import numpy as np

In [22]:
df = pd.read_csv('data/2ch_posts.csv')

In [24]:
BATCH_SIZE = 4
system = f"""
    You are given exactly {BATCH_SIZE} posts from a special forum where people post their problems and ask for advice.
    The forum is called "2ch" and it is very popular in Russia.
    Your task is to determine if this message is *toxic* or not.
    Your response will be interpreted with an API that expects exactly {BATCH_SIZE} numbers separated by a space, one for each post: '1' if the message is toxic or '0' otherwise. Dont write anything else.
"""

client = OpenAI(api_key="")

mapping = []
cur_posts = []
for post in tqdm(df['text'][2006:]): 
    cur_posts.append(post)
    if len(cur_posts) >= BATCH_SIZE:
        for i in range(5):
            messages = [{"role": "system", "content": system}]
            for i in cur_posts:
                messages.append({"role": "user", "content": i})
            try:
                completion = client.chat.completions.create(
                  model="gpt-4",
                  messages=messages,
                )
            except RateLimitError as e:
                print(e.message)
                time.sleep(10)
                continue
            if len(completion.choices) >= 1 and isinstance(completion.choices[0].message.content, str):
                response = completion.choices[0].message.content
                # print(response)
                spl = response.strip().replace(',', ' ').replace('  ', ' ').replace('\n', ' ').strip()
                spl = spl.split(' ')
                if len(spl) != BATCH_SIZE:
                    # print("Response length is not equal to batch size")
                    continue
                for i in range(BATCH_SIZE-1, -1, -1):
                    if spl[i].strip() not in ['0', '1']:
                        # print("Response is not a number")
                        continue
                    mapping.append((cur_posts[i], int(spl[i])))
                    # print(cur_posts[i], ": ", spl[i], sep='')
                    del cur_posts[i]
                if len(cur_posts) == 0:
                    break
            else:
                print(f"Response len {len(completion.choices)}")
                continue
        cur_posts = []

  2%|▏         | 206/13344 [01:18<2:01:56,  1.80it/s]

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-sB3Q6X5Gn6yDbFGorPNcquJG on tokens_usage_based per min: Limit 10000, Used 9193, Requested 1469. Please try again in 3.971s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens_usage_based', 'param': None, 'code': 'rate_limit_exceeded'}}


  3%|▎         | 396/13344 [03:37<2:36:24,  1.38it/s]

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-sB3Q6X5Gn6yDbFGorPNcquJG on tokens_usage_based per min: Limit 10000, Used 9279, Requested 798. Please try again in 462ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens_usage_based', 'param': None, 'code': 'rate_limit_exceeded'}}


  5%|▌         | 692/13344 [07:29<2:36:46,  1.34it/s]

...

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-sB3Q6X5Gn6yDbFGorPNcquJG on tokens_usage_based per min: Limit 10000, Used 9216, Requested 923. Please try again in 834ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens_usage_based', 'param': None, 'code': 'rate_limit_exceeded'}}


 15%|█▌        | 2010/13344 [23:25<2:51:11,  1.10it/s]

Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


 15%|█▌        | 2011/13344 [24:06<2:15:51,  1.39it/s]


KeyboardInterrupt: 

In [17]:
mapping

[('сказать-то что хотел?', 0),
 ('всю жизнь был нищим. на работу никуда не брали никогда. жил с поехавшей матерью веруншей на 35к в месяц. на словах у неё всемогущий бог лечит даже рак, а на деле когда температура подскочила гонит в аптеку за антибиотиками. она открыто называла себя "рабой божьей". надо ли говорить, что уважающий себя человек так никогда не назовёт себя? а кто не хочет как она молиться на коленях и каяться и имеет чувство собственного достоинства, тот грешник и это гордыня. пример разговора:-бог есть?-да.-он людей любит?-да.-мы страдаем и наша жизнь это муки, но если бы он нас любил, то устроил на счастливую жизнь и радость. но этого нет. значит, бога нет.-нет, он есть.-ну тогда он уебок и мразь. ведь сейчас кто-то умирает от рака или незаслуженно сидит в тюрьме, а он нихуя не делает.-не говори так, в ад попадёшь. он любящий!-он нихуя не сделал для нас. -она умер за наши грехи и подарит жизнь вечную.  вот и весь разговор! это непробиваемое тупое покорное рабское чмо с 

In [20]:
df = pd.DataFrame(mapping, columns=['text', 'toxic'])

In [21]:
df.to_csv('data/2ch_labeled_gpt4.csv', index=True, index_label='id')